In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re 
from sklearn.feature_extraction.text import CountVectorizer
from datetime import datetime
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from textblob import TextBlob
from nltk.util import ngrams


data = pd.read_csv('bq-results-20190608-151444-lhfwg26di18j.csv')

In [2]:
data.shape

(500000, 7)

In [3]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 7 columns):
parent_id       500000 non-null object
subreddit_id    500000 non-null object
subreddit       500000 non-null object
created_utc     500000 non-null int64
author          500000 non-null object
score           500000 non-null int64
body            499996 non-null object
dtypes: int64(2), object(5)
memory usage: 26.7+ MB
None


In [4]:
data.tail(5)

,parent_id,subreddit_id,subreddit,created_utc,author,score,body
499995,t3_aunzqy,t5_x5zpe,MadridBarcelonaLive,1551140268,[deleted],1,[removed]
499996,t3_aunzqy,t5_x5zpe,MadridBarcelonaLive,1551140453,[deleted],1,[removed]
499997,t3_aunzqy,t5_x5zpe,MadridBarcelonaLive,1551140514,[deleted],1,[removed]
499998,t3_aunzqy,t5_x5zpe,MadridBarcelonaLive,1551140578,[deleted],1,[removed]
499999,t3_aunzqy,t5_x5zpe,MadridBarcelonaLive,1551140622,[deleted],1,[removed]


In [75]:
data[data.subreddit=="nba"].sample(10)

,parent_id,subreddit_id,subreddit,created_utc,author,score,body
130283,t3_amjoqf,t5_2qo4s,nba,1549152624,DifficultDisplay1,13,"Question, can someone tell me how many years e..."
129454,t3_anxlpr,t5_2qo4s,nba,1549498595,Karakurizer,5,Nah the Wizards like their current purgatory
126685,t3_as36gd,t5_2qo4s,nba,1550535626,DrumzRUs,1,Curry's way more iconic
126599,t3_ankgtn,t5_2qo4s,nba,1549411281,Karakurizer,1,They had to deal Cov for him. He will stay
129227,t1_egp0dmi,t5_2qo4s,nba,1550448112,rando1817,4,I may be petty and that’s not my only reason f...
126301,t1_efu1hv5,t5_2qo4s,nba,1549412327,retrohhh999,1,I want JaMychal Green too please
125161,t3_apabvg,t5_2qo4s,nba,1549844494,carofdoom123,225,The Boston Acquaintances
128383,t1_efzg0wi,t5_2qo4s,nba,1549585582,mkz419,2,only because it would be too obvious at that p...
129959,t3_aoabqf,t5_2qo4s,nba,1549584435,MyLadySansa,8,"So, basically Bron gets first crack at both ro..."
124808,t1_egoz9vr,t5_2qo4s,nba,1550448045,ineedmorechainsaws,69,You're right that they're no different. The i...


In [8]:
len(data[data.subreddit=="nba"])

6307

In [22]:
print (data.groupby('subreddit')['body'].nunique().sort_values(ascending=False).head())

subreddit
AskReddit    23861
politics      7724
nba           5794
nfl           5157
funny         4093
Name: body, dtype: int64


In [23]:
data.groupby('subreddit').agg('count').sort_values('body', ascending=False)

,parent_id,subreddit_id,created_utc,author,score,body
subreddit,,,,,,
AskReddit,25706,25706,25706,25706,25706,25706
politics,8845,8845,8845,8845,8845,8845
nba,6307,6307,6307,6307,6307,6307
nfl,5595,5595,5595,5595,5595,5595
funny,4544,4544,4544,4544,4544,4544
FortNiteBR,4022,4022,4022,4022,4022,4022
The_Donald,3889,3889,3889,3889,3889,3887
dankmemes,3809,3809,3809,3809,3809,3809
apexlegends,3750,3750,3750,3750,3750,3750


### change created_utc

In [5]:
data.created_utc[1]

1549239892

In [6]:
parsed_date = datetime.utcfromtimestamp(data.created_utc[1])
parsed_date

datetime.datetime(2019, 2, 4, 0, 24, 52)

In [7]:
year = parsed_date.year
month = parsed_date.month
day = parsed_date.day
hour = parsed_date.hour
minute = parsed_date.minute
second = parsed_date.second
second

52

In [8]:
data.created_utc.apply(lambda x:datetime.utcfromtimestamp(x))

0        2019-02-03 00:20:05
1        2019-02-04 00:24:52
2        2019-02-14 00:04:21
3        2019-02-03 00:14:05
4        2019-02-12 00:13:01
5        2019-02-03 00:14:54
6        2019-02-14 00:17:10
7        2019-02-15 00:02:30
8        2019-02-22 00:10:10
9        2019-02-19 00:19:09
10       2019-02-28 00:01:08
11       2019-02-09 00:07:08
12       2019-02-19 00:24:55
13       2019-02-03 00:26:36
14       2019-02-01 00:01:12
15       2019-02-02 00:08:24
16       2019-02-16 00:11:26
17       2019-02-18 00:03:03
18       2019-02-21 00:08:12
19       2019-02-02 00:09:14
20       2019-02-02 00:13:50
21       2019-02-21 00:18:31
22       2019-02-12 00:18:14
23       2019-02-07 00:29:27
24       2019-02-03 00:05:14
25       2019-02-23 00:28:53
26       2019-02-07 00:26:42
27       2019-02-02 00:01:43
28       2019-02-10 00:21:25
29       2019-02-04 00:17:32
                 ...        
499970   2019-02-26 00:14:24
499971   2019-02-26 00:14:43
499972   2019-02-26 00:14:45
499973   2019-

### Removing stop words with NLTK & counting each word 

In [9]:
set(stopwords.words('english')) 

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [10]:
data.body[13]

'READ THIS MESSAGE! We require a minimum account-age and karma.  ***These minimums are not disclosed***.  No exceptions will be made.  Messaging mods for the minimums will result in a 30-day ban.\n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/nsfw) if you have any questions or concerns.*'

In [11]:
# data.body[13] = [re.sub(r"[^a-zA-Z0-9]+", ' ', k) for k in data.body[13].lower().split("\n")]
data.body[13] = re.sub(r'\W+', ' ', data.body[13].lower())
data.body[13]

/Users/juliekim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


'read this message we require a minimum account age and karma these minimums are not disclosed no exceptions will be made messaging mods for the minimums will result in a 30 day ban i am a bot and this action was performed automatically please contact the moderators of this subreddit message compose to r nsfw if you have any questions or concerns '

In [40]:
example_sent = data.body[13]
stop_words = set(stopwords.words('english')) 
word_tokens = word_tokenize(data.body[13]) 
filtered_sentence = [w for w in word_tokens if not w in stop_words] 
filtered_sentence = []  
for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w)   
print(word_tokens) 
print(filtered_sentence) 

words = filtered_sentence
counts = dict()
for word in words:
    if word in counts:
        counts[word] += 1

    else:
        counts[word] = 1
print(counts)

['read', 'this', 'message', 'we', 'require', 'a', 'minimum', 'account', 'age', 'and', 'karma', 'these', 'minimums', 'are', 'not', 'disclosed', 'no', 'exceptions', 'will', 'be', 'made', 'messaging', 'mods', 'for', 'the', 'minimums', 'will', 'result', 'in', 'a', '30', 'day', 'ban', 'i', 'am', 'a', 'bot', 'and', 'this', 'action', 'was', 'performed', 'automatically', 'please', 'contact', 'the', 'moderators', 'of', 'this', 'subreddit', 'message', 'compose', 'to', 'r', 'nsfw', 'if', 'you', 'have', 'any', 'questions', 'or', 'concerns']
['read', 'message', 'require', 'minimum', 'account', 'age', 'karma', 'minimums', 'disclosed', 'exceptions', 'made', 'messaging', 'mods', 'minimums', 'result', '30', 'day', 'ban', 'bot', 'action', 'performed', 'automatically', 'please', 'contact', 'moderators', 'subreddit', 'message', 'compose', 'r', 'nsfw', 'questions', 'concerns']
{'read': 1, 'message': 2, 'require': 1, 'minimum': 1, 'account': 1, 'age': 1, 'karma': 1, 'minimums': 2, 'disclosed': 1, 'exception

### Clean comments function 

In [85]:
def clean(sentence):
    sentence = re.sub(r'\W+', ' ', sentence.lower())
    return sentence 

In [103]:
test1 = "I'm going to swim, and then eat. How about you?"
clean(test1)
# num_word = group.body.apply(lambda x: word_count(x))
clean_df = group.body.apply(lambda x: clean(x))
clean_df= pd.DataFrame(clean_df)


In [150]:
clean_df.head(5)

,body
0,u w u so far it goes to mk4 but i dont have t...
1,i think that s a good idea her constant bad us...
2,hey there looks like you re not using our stan...
3,ok no problem
4,well deserved if my girls look that good at 40...


### Remove stop words function 

In [135]:
def token_func(sentence):
    stop_words = set(stopwords.words('english')) 
  
    word_tokens = word_tokenize(sentence) 
  
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
    filtered_sentence = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)
            
    return filtered_sentence

In [157]:
# test1 = "I'm going to swim, and then eat. How about you?"
# token_func(clean(test1))
token_df = group.body.apply(lambda x: token_func(clean(x)))
token_df= pd.DataFrame(token_df)
token_df.head()


,body
0,"[u, w, u, far, goes, mk4, dont, drawings, mk3, 4]"
1,"[think, good, idea, constant, bad, use, photos..."
2,"[hey, looks, like, using, standard, flair, for..."
3,"[ok, problem]"
4,"[well, deserved, girls, look, good, 40, smilin..."


### Word count function 

In [15]:

def word_count(sentence):
    sentence = re.sub(r'\W+', ' ', sentence.lower())
    counts = dict()
    
    ### Removing stopwords
    stop_words = set(stopwords.words('english')) 
  
    word_tokens = word_tokenize(sentence) 
  
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
    filtered_sentence = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    
    ###Word count
    words = filtered_sentence

    for word in words:
        if word in counts:
            counts[word] += 1

        else:
            counts[word] = 1
    return counts



### Group by Subreddit 

In [14]:
# pgroup = data.groupby(['subreddit'], as_index = False)
# new_data = pgroup.apply(lambda x: x['body'].unique())

In [15]:
# data[['subreddit', 'body']].groupby('subreddit')
# data['combined'] = data['body'].values.tolist()

In [11]:
group = (data.groupby('subreddit')['body'].apply(lambda x: ','.join(set(x.dropna()))).reset_index())

In [12]:
group.head(8)

,subreddit,body
0,0w0,so far it goes to MK4 but i dont have the draw...
1,1000ccplus,"What did it say lol,Her constant (bad) use of ..."
2,1000daysofpractice,"Music 27\n\ntwo hours, played the first page o..."
3,100ThievesApparel,ok no problem
4,100sexiest,Well deserved....if my girls look that good at...
5,100thieves,Like I'm all for picking unconventional lineup...
6,100yearsago,"Yup, it made think of my little brother wearin..."
7,101Wicca,it's fine. We accept everyone from everywhere ...


In [18]:
group.shape

(20103, 2)

In [19]:
group.body[6]

'Yup, it made think of my little brother wearing my grandfather’s WWl helmet when he was a small boy.,Strike on the Tube. Nothing ever changes. ,what a different world had the US gone in on the German side....and there was talk'

In [20]:
# new_data

In [21]:
# new_data[6]


### word count for each subreddit 

In [17]:
num_word = group.body.apply(lambda x: word_count(x))


In [18]:
num_word= pd.DataFrame(num_word)
num_word.head()

,body
0,"{'far': 1, 'goes': 1, 'mk4': 1, 'dont': 1, 'dr..."
1,"{'say': 1, 'lol': 1, 'constant': 1, 'bad': 1, ..."
2,"{'music': 1, '27': 1, 'two': 1, 'hours': 1, 'p..."
3,"{'ok': 1, 'problem': 1}"
4,"{'well': 1, 'deserved': 1, 'girls': 1, 'look':..."


In [19]:
df = pd.concat([group['subreddit'], num_word['body']], axis=1, keys=['Subreddit', 'Values'])
df['Values'] = df['Values'].map(lambda t: sorted(t.items(), key = lambda x: x[1],reverse=True))
# df.sort_values(['Values'], ascending=[False])
df.sample(15)


,Subreddit,Values
2499,CultOfGuac,"[(ahaha, 1), (amp, 1), (x200b, 1), (yaaas, 1)]"
7170,OldPhotosInRealLife,"[(side, 2), (r, 1), (1healthbar, 1), (like, 1)..."
5489,KingBuffalo,"[(rock, 2), (favorite, 2), (well, 2), (music, ..."
5260,JessicaNigri,"[(cosplay, 2), (think, 2), (ever, 2), (general..."
19274,u_mobilemechanicrgv,"[(tx, 64), (towing, 42), (edinburg, 36), (mcal..."
11264,alpha,"[(true, 1), (alpha, 1), (males, 1), (upvote, 1..."
17395,rs2vietnam,"[(game, 13), (gt, 10), (gameplay, 6), (like, 5..."
16770,pkmntcgreferences,"[(cards, 2), (great, 2), (would, 2), (recommen..."
8242,RedditSilverRobot,"[(redditbronze, 1)]"
19088,u_TTVunknown_phantomYT,"[(would, 1), (like, 1), (join, 1)]"


In [21]:
df[df.Subreddit == 'nba'].Values.tolist()

[[('like', 606),
  ('would', 513),
  ('team', 510),
  ('get', 421),
  ('lebron', 379),
  ('think', 373),
  ('good', 327),
  ('year', 325),
  ('one', 323),
  ('even', 317),
  ('game', 311),
  ('players', 308),
  ('better', 302),
  ('player', 300),
  ('nba', 274),
  ('lakers', 273),
  ('gt', 270),
  ('still', 250),
  ('got', 249),
  ('lol', 249),
  ('really', 246),
  ('people', 245),
  ('time', 234),
  ('trade', 227),
  ('know', 223),
  ('best', 219),
  ('ad', 217),
  ('teams', 216),
  ('play', 215),
  ('season', 212),
  ('could', 206),
  ('see', 205),
  ('much', 203),
  ('going', 197),
  ('3', 194),
  ('right', 193),
  ('make', 191),
  ('last', 191),
  ('want', 188),
  ('go', 180),
  ('also', 180),
  ('way', 177),
  ('guys', 169),
  ('years', 167),
  ('star', 167),
  ('well', 167),
  ('said', 160),
  ('5', 157),
  ('league', 156),
  ('first', 154),
  ('bad', 151),
  ('say', 150),
  ('shit', 147),
  ('man', 147),
  ('point', 145),
  ('yeah', 139),
  ('guy', 139),
  ('1', 139),
  ('great'

### N-grams

In [72]:
group['body'][7]

"it's fine. We accept everyone from everywhere even outside the country. The only thing is you would have to come to our location for your dedications and the elevations after that."

In [74]:
from nltk.util import ngrams
from textblob import TextBlob

sample_text = re.sub(r'\W+', ' ', group['body'][7].lower())
TextBlob(sample_text).ngrams(3)

[WordList(['it', 's', 'fine']),
 WordList(['s', 'fine', 'we']),
 WordList(['fine', 'we', 'accept']),
 WordList(['we', 'accept', 'everyone']),
 WordList(['accept', 'everyone', 'from']),
 WordList(['everyone', 'from', 'everywhere']),
 WordList(['from', 'everywhere', 'even']),
 WordList(['everywhere', 'even', 'outside']),
 WordList(['even', 'outside', 'the']),
 WordList(['outside', 'the', 'country']),
 WordList(['the', 'country', 'the']),
 WordList(['country', 'the', 'only']),
 WordList(['the', 'only', 'thing']),
 WordList(['only', 'thing', 'is']),
 WordList(['thing', 'is', 'you']),
 WordList(['is', 'you', 'would']),
 WordList(['you', 'would', 'have']),
 WordList(['would', 'have', 'to']),
 WordList(['have', 'to', 'come']),
 WordList(['to', 'come', 'to']),
 WordList(['come', 'to', 'our']),
 WordList(['to', 'our', 'location']),
 WordList(['our', 'location', 'for']),
 WordList(['location', 'for', 'your']),
 WordList(['for', 'your', 'dedications']),
 WordList(['your', 'dedications', 'and']),


### Sentiment Analysis 

In [ ]:
def get_sentiment(self, word):
    '''
    Utility function to determine sentiment of passed word 
    using textblob's sentiment method
    '''
    
    # create TextBlob object of passed tweet text 
    analysis = TextBlob(self.clean(word)) 
    
    if analysis.sentiment.polarity > 0:
        return "positive"
    elif analysis.sentiment.polarity ==0:
        return "neutral"
    else:
        return "negative"

In [105]:
sample_size = 10000

def sentiment_func(x):
    sentiment = TextBlob(x)
    x['polarity'] = sentiment.polarity
    x['subjectivity'] = sentiment.subjectivity
    return x

sample = clean_df(sample_text).apply(sentiment_func, axis=1)

TypeError: 'DataFrame' object is not callable

### VADER - Valence Aware Dictionary and sEntiment Reasoner 

In [109]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [110]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))

In [122]:
sentiment_analyzer_scores("the service isn't good!")

the service isn't good!----------------- {'neg': 0.473, 'neu': 0.527, 'pos': 0.0, 'compound': -0.4015}


In [121]:
sentiment_analyzer_scores("the service isn't extremely good!")

the service isn't extremely good!------- {'neg': 0.422, 'neu': 0.578, 'pos': 0.0, 'compound': -0.4432}


In [114]:
sentiment_analyzer_scores("The food here isn’t really all that great")


The food here isn’t really all that great {'neg': 0.0, 'neu': 0.616, 'pos': 0.384, 'compound': 0.6557}


### Tokenization using spaCy

In [ ]:
import spacy
# Load the en_core_web_sm model
nlp = spacy.load('en_core_web_sm')

### TF-IDF (Term Frequency - Inverse Document Frequency)

In [127]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = CountVectorizer()
# Generate matrix of wrd vectors (example)
bow_matrix = vectorizer.fit_transform(clean_df['body']) 
print(bow_matrix.shape)

(20103, 235119)


In [154]:
bow_matrix_2 = vectorizer.fit_transform(token_df['body']) 
print(bow_matrix_2.shape)

AttributeError: 'list' object has no attribute 'lower'

In [126]:
clean_df.head()

,body
0,u w u so far it goes to mk4 but i dont have t...
1,i think that s a good idea her constant bad us...
2,hey there looks like you re not using our stan...
3,ok no problem
4,well deserved if my girls look that good at 40...


### score analysis 

In [27]:
pd.DataFrame(data.groupby('subreddit')['score'].agg('mean').sort_values().head(20))

,score
subreddit,
EmilyBloom,-62.000000
karleegrey,-27.000000
Cumberbitches,-26.000000
badpsychology,-25.000000
Purrito,-22.000000
FortniteAfterDark,-21.000000
Megturney,-17.666667
anriokita,-16.500000
MarvelCringe,-16.000000


In [28]:
pd.DataFrame(data.groupby('subreddit')['score'].agg('mean').sort_values(ascending=False).head(20))

,score
subreddit,
KamikazeByWords,266.000000
menwritingwomen,174.750000
TheDepthsBelow,162.800000
preyingmantis,140.666667
likeus,122.666667
arresteddevelopment,111.000000
u_RedditCancerBot420,101.000000
watchplantsgrow,98.000000
BreadStapledToTrees,85.000000


### Vader analysis 

In [30]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
#     print("{}".format(str(score)))
    return score
top_20 = data.sort_values('subreddit').reset_index()
top_20['senti']= data['body'].map(lambda t: sentiment_analyzer_scores(str(t))) 
top_20['senti_neu'] = top_20.senti.apply(lambda x: x.get('neu'))
top_20['senti_pos'] = top_20.senti.apply(lambda x: x.get('pos'))
top_20['senti_neg'] = top_20.senti.apply(lambda x: x.get('neg'))

In [31]:
pd.DataFrame(top_20.groupby('subreddit')['senti_pos'].agg('mean').sort_values(ascending=False).head(20))

,senti_pos
subreddit,
Callieism,1.0
VTCC,1.0
ProdoGlassHeads,1.0
exteenagers,1.0
MHOCHolyrood,1.0
FLMMJDISPENSARIES,1.0
GoldReplies,1.0
babybigcatgifs,1.0
HungryButts,1.0


In [33]:
pd.DataFrame(top_20.groupby('subreddit')['senti_neg'].agg('mean').sort_values(ascending=False).head(50))

,senti_neg
subreddit,
sciencestudioyt,1.0000
republicans,1.0000
rprogramming,1.0000
rarebooks,1.0000
castration,1.0000
BattleForTheGrid,1.0000
celebritylookalike,1.0000
audacity,1.0000
leahgotti,1.0000


In [34]:
pd.DataFrame(top_20.groupby('subreddit')['senti_neu'].agg('mean').sort_values(ascending=False).head(20))

,senti_neu
subreddit,
zztails,1.0
bookporn,1.0
budgetwise,1.0
bsv,1.0
bspwm,1.0
brokenthoughts,1.0
brissabreezy,1.0
brazilianjiujitsu,1.0
bravelydefault,1.0


In [41]:
pd.DataFrame(top_20[['subreddit','senti_neg', 'senti_neu', 'senti_pos']]).sample(20)


,subreddit,senti_neg,senti_neu,senti_pos
374661,mildlyinfuriating,0.072,0.868,0.060
50460,AskWomen,0.066,0.797,0.137
22863,AskOuija,0.074,0.858,0.068
477018,u_hotelsworldwide,0.000,1.000,0.000
31969,AskReddit,0.114,0.549,0.338
403553,opieandanthony,0.000,1.000,0.000
293909,dataisbeautiful,0.000,0.813,0.187
62019,BravoRealHousewives,0.000,1.000,0.000
228631,Tinder,0.000,0.896,0.104
407672,pcmasterrace,0.108,0.829,0.063
